# Landscape Grid

## Setup

### Ecoscope

In [ ]:
ECOSCOPE_RAW = "https://raw.githubusercontent.com/wildlife-dynamics/ecoscope/master"

%pip install \
    'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v1.8.29' &> /dev/null

In [ ]:
import os
import sys

import geopandas as gpd

import ecoscope

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

## Read Sample AOI

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/landscape_grid.gpkg",
    os.path.join(output_dir, "landscape_grid.gpkg"),
)

AOI = gpd.read_file(os.path.join(output_dir, "landscape_grid.gpkg"), layer="AOI")
print(AOI.crs)

AOI.explore()

## Read Sample Grid

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/landscape_grid.gpkg",
    os.path.join(output_dir, "landscape_grid.gpkg"),
)

exist_grid = gpd.read_file(os.path.join(output_dir, "landscape_grid.gpkg"), layer="existing_grid_1_km")
print(exist_grid.crs)

exist_grid.explore()

### Remove Empty Geometries in Grid

In [ ]:
def clean_geodataframe(gdf):
    return gdf.loc[(~gdf.geometry.isna()) & (~gdf.geometry.is_empty)]


exist_grid = clean_geodataframe(exist_grid)

any(exist_grid.geometry.isna())

## Scenario 1: Create a New Landscape Grid based on AOI

In [ ]:
new_grid_5_km = ecoscope.base.utils.create_meshgrid(
    aoi=AOI.unary_union,  # Union of input geometries
    in_crs=AOI.crs,
    out_crs=4326,
    xlen=5000,
    ylen=5000,
    return_intersecting_only=True,  # Whether only grid cells intersecting the AOI will be returned
    align_to_existing=None,
)

print(new_grid_5_km.crs)

new_grid_5_km.explore()

Export:

In [ ]:
new_grid_5_km.to_file(
    os.path.join(output_dir, "out_landscape_grid.gpkg"),
    layer="new_grid_5_km",
    driver="GPKG",
)

## Scenario 2: Align Grid Cells to Existing Grid to Cover AOI

In [ ]:
new_grid_1_km_aligned = ecoscope.base.utils.create_meshgrid(
    aoi=AOI.unary_union,  # Union of input geometries
    in_crs=AOI.crs,
    out_crs=4326,
    xlen=1000,
    ylen=1000,
    return_intersecting_only=True,  # Whether only grid cells intersecting the AOI will be returned
    align_to_existing=exist_grid,
)

print(new_grid_1_km_aligned.crs)

new_grid_1_km_aligned.explore()

Export

In [ ]:
new_grid_1_km_aligned.to_file(
    os.path.join(output_dir, "out_landscape_grid.gpkg"),
    layer="new_grid_1_km_aligned",
    driver="GPKG",
)